# DS 2500: Data Wrangling<br>Module Assignment 3
March 1, 2022

Student: Alexander Shahramanyan<br>
Instructor: Professor Marina Kogan

## Introduction
Cryptocurrencies gained significant popularity over the last years and has gathered big communities of traders. In this assignment, I'll use the CoinAPI to get Bitcoin price data and will try to get some insights from it.

## Questions:
1. Is the Bitcoin price data (which is timeseries, of course) stationary (it should be non-stationary, however, I'll use a statistical test to ensure that)?
2. Are number of trades or volume traded correlated with Bitcoin prices? 
3. Is there a correlation between Covid cases and Bitcoin price?
<hr>
0 - Is there a significant difference between the Bitcoin prices of any two consecutive weeks as compared to others?

## Dataset Description:
**Dataset 1: Bitcoin Prices**: from CoinAPI. Contains hourly data on Bitcoin prices for 2020: open time, open and close, high and low, prices, volume traded, and trades count.
<br>

*Columns*:<br>
7 columns
- time_open - datetime
- price_open - float
- price_high - float
- price_low - float
- price_close - float
- volume_traded - float
- trades_count - int

*Number of entries (rows)*: 8759 (366\*24=8784, however API fails to provide some data)<br><hr>

**Dataset 2: Covid Cases**: from John Hopkins University Covid tracking website (via github). Contains daily information on cumulative Covid cases per province/state and country/region.
*Columns*:<br>
 772 columns
- Province/State - string
- Country/Region - string
- Lat (latitude) - float
- Long (longitude) - float
- 1/22/20 - int (cumulative Covid cases)
- .......
- 2/27/22 - int (cumulative Covid cases)

*Number of entries (rows)*: 284

## Setup
First, I import all the libraries I will need throughout the notebook and set the plotting parameters for better visualizations.

In [1]:
import json
import requests
import pandas as pd
import time
import datetime as dt
import json
from scipy.stats import stats, pearsonr
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns
import textwrap

# Plotting settings
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 18
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['font.family'] = 'serif'

CoinAPI-Keys:<br>
 - 7C42C2B0-B30F-4988-9B5C-6F6EE0237D3E  <br>
 - 49BAD700-37F7-4776-9A80-E95274FE737A  <br>
 - D6F38463-FC87-4C68-AF3A-FBCC83F2A57D  <br>
 - E87012DB-899D-41CD-A2D7-CB6510046225  <br>
 - 1BB08CB1-10A5-44CF-9B48-72245E12F41D  <br>
 - 831F56DE-37FD-4D77-B4FC-FE72108716A0

## Get Data

Next, I define a function that gets data from CoinAPI. Since there is a limit on the amount of data that that can be get per request, I will call this function repeatedly in order to get hourly data for a full year.

In [2]:
def get_data(symbol="BINANCE_SPOT_BTC_USDT", start_date="2020-01-01", end_date="2020-12-31", step="1HRS", convert_date_times=True):
    """
    Gets and returns the price data of the symbol, for the given time interval and frequence.
    
    Keyword parameters:
        symbol            - the symbol whose price data should be downloaded.
                           Default: "BINANCE_SPOT_BTC_USDT"
                
        start_date         - the start date.
                           Default: "2020-01-01"
        
        end_date           - the end date.
                           Default: "2020-12-31"
        
        step               - the frequency of the data.
                           Possible values: 1SEC, 2SEC, 3SEC, 4SEC, 5SEC, 6SEC, 10SEC, 15SEC, 20SEC, 30SEC,
                                            1MIN, 2MIN, 3MIN, 4MIN, 5MIN, 6MIN, 10MIN, 15MIN, 20MIN, 30MIN,
                                            1HRS, 2HRS, 3HRS, 4HRS, 6HRS, 8HRS, 12HRS,
                                            1DAY, 2DAY, 3DAY, 5DAY, 7DAY, 10DAY.
                           Default: "1HRS"
            
        convert_date_times - if true, will convert the start and end dates from
                           'YYYY-MM-DD' into a datetime object, otherwise considers
                           the dates are datetimes.
                           Default: True
    Returns:
        A list of json objects witht the price data.
    """
    # Header for the request
    headers = {"X-CoinAPI-Key" : "7C42C2B0-B30F-4988-9B5C-6F6EE0237D3E"}
    
    # List for the JSON objects
    responses = []
    
    # Convert the start and end dates into datetimes if needed
    if convert_date_times:
        start_date = dt.datetime.combine(dt.date.fromisoformat(start_date), dt.time(0, 0, 0, 0))
        end_date = dt.datetime.combine(dt.date.fromisoformat(end_date), dt.time(23, 59, 0, 0))
    
    # Convert the end date into the required string format to use in the link
    end_date_str = end_date.strftime("%Y-%m-%dT%H:%M:%S")
    
    if step[-3:] == "SEC":
        window = dt.timedelta(seconds = int(step[:-3])*100)
    elif step[-3:] == "MIN":
        window = dt.timedelta(minutes = int(step[:-3])*100)
    elif step[-3:] == "HRS":
        window = dt.timedelta(hours = int(step[:-3])*100)
    elif step[-3:] == "DAY":
        window = dt.timedelta(days = int(step[:-3])*100)
    else:
        raise ValueError("Illegal step argument.")

    # While there still is data to download
    while start_date <= end_date:
        # Convert the current start time into the requiored string format to use in the link
        start = start_date.strftime("%Y-%m-%dT%H:%M:%S")
        
        # Add the windows to the start date
        start_date += window
        
        # Create the url
        url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/history?period_id={step}&time_start={start}&time_end={end_date_str}"
        
        # Send the request and store the response
        response = requests.get(url, headers=headers)
        
        # Not OK API responses
        if response.status_code != 200:
            if response.status_code == 400:
                print("Bad Request -- There is something wrong with your request.")
            elif response.status_code == 401:
                print("Unauthorized -- Your API key is wrong.")
            elif response.status_code == 403:
                print("Forbidden -- Your API key doesnt't have enough privileges to access this resource.")
            elif response.status_code == 429:
                print("Too many requests -- You have exceeded your API key rate limits.")
            elif response.status_code == 550:
                print("No data -- You requested specific single item that we don't have at this moment.")
            
            print("Exiting function.")
            
            return responses
        
        # Add the JSON objects to the list
        responses += response.json()
        
        # Wait for a second
        time.sleep(1)
        
    return responses

I will also need to make a dataframe from the JSON objects. For that, I will create another function.

In [3]:
def to_df(responses):
    """
    Creates a dataframe from the JSON objects and return. Converts the time columns into datetime.
    
    Keyword parameters:
        responses - a list with JSON objects.
        
    Returns:
        A dataframe created from the JSON objects.
    """
    # Initialize a dataframe from the responses
    df = pd.DataFrame(responses)
    
    # Change the types of the time columns to datetime
    for col in [x for x in df.columns if "time" in x]:
        df[col] = pd.to_datetime(df[col])
    
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Sort the data by time_open
    df.sort_values('time_period_start', inplace=True)
    
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
# Download the data
responses = get_data(symbol="BINANCE_SPOT_BTC_USDT", start_date="2020-01-01", end_date="2020-12-31",\
                     step="1HRS", convert_date_times=True)

In [ ]:
# Create a dataframe from the downloaded data
df = to_df(responses)

## Dataset Check

In [ ]:
# Load the dataset
# df = pd.read_csv("BTCUSDT_2020.csv", index_col=0)

We should have 366*24 = 8784 rows in the dataframe. I'll check that using df.shape.

In [ ]:
# Check dataset shape
df.shape

As we see, we have only 8759 rows. There seems to be some error with the CoinAPI, I'll try to get the missing data again. Now, I'll check the top and bottom of the data, to ensure the start and end dates are correct.

In [ ]:
# Check the top of the data
df.head()

In [ ]:
# Check the bottom of the data
df.tail()

In [ ]:
# Check the number of rows in the dataframe
len(df)

In [ ]:
# Check null values
df.isnull().sum()

I'll only keep only one time data column. The others are unnecessary.

In [ ]:
# Keep only one time data column
df = df[['time_period_start', 'price_open', 'price_high', 'price_low',\
         'price_close', 'volume_traded', 'trades_count']]

In [ ]:
# Rename the time data column
df.rename(columns = {'time_period_start':'time_open'}, inplace = True)

Try to download the missing data again.

In [ ]:
# Try to download the missing data again using the API
for row in range(1, len(df.index)):
    # If the difference between two consecutive values is not 1, download the data between 
    if abs(df.time_open[row] - df.time_open[row-1] - dt.timedelta(hours=1)) > dt.timedelta(minutes=5):
        start_date = df.time_open[row-1] + dt.timedelta(hours=1)
        end_date = df.time_open[row]
        
        responses = get_data(start_date=start_date, end_date=end_date, step="1HRS", convert_date_times=False)

        # If nothing was returned from the request, continue
        if responses == []:
            continue
        
        # Print the JSON objects returned
        print(responses)
        
        # Create a dataframe from the new JSON objects
        df_add = to_df(responses)
    
        df_add = df_add[['time_period_start', 'price_open', 'price_high', 'price_low',\
             'price_close', 'volume_traded', 'trades_count']]
        
        df_add.rename(columns = {'time_period_start':'time_open'}, inplace = True)

        # Append the new data to the dataframe
        df = df.append(df_add, ignore_index = True)
        
# Drop duplicates
df.drop_duplicates(inplace=True)
    
# Sort the data by time_open
df.sort_values('time_open', inplace=True)
    
# Reset index
df.reset_index(drop=True, inplace=True)

In [ ]:
# Check the number of rows in the dataframe
len(df)

As we see, no new rows were added. There seems to be some problem with the API. Since only 25 entries are missing, there is no need to worry. I will now plot the priceline (close prices) to see if there is any trend.

In [ ]:
# Priceline
ax = sns.lineplot(x="time_open", y="price_close", data=df, color='#748B75')

# Set y-axis label
ax.set_ylabel("Price ($)")

# Set x-axis label
ax.set_xlabel("Date")

# Set x limits
ax.set_xlim(df.iloc[0].time_open, df.iloc[-1].time_open)

# Set y limits
ax.set_ylim(df.price_close.min()*.92, df.price_close.max()*1.08)

# Title
ax.set_title("BTC Price Over Time")

# Show the plot
plt.show()

There is a steady increaseing trend in the close prices over the year.

## Q1: Stationary or Non-Stationary

Here, I'll check if the price data is stationary or not. Stationarity implies that taking consecutive samples of data with the same size should have identical covariances regardless of the starting point. The Bitcoin price is very volatile and has changed significantly and rapidly over time. So it should not be stationary. To check that I'll conduct a Dickey-Fuller Test.

In [ ]:
def adf_test(timeseries):
    """
    Calculates and prints the results of Dickey-Fuller test on the given timeseries.
    
    Keyword parameters:
        timeseries - the timeseries, on which the Dickey-Fuller test should be calculated.
    """
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
dftest = adf_test(df['price_close'])

The p-value is greater than 0.05, the timeseries is non-stationary.

## Q2: Correlations Between Price and Trades count

I'll calculate correlation between a couple of variables.

In [ ]:
corr, _ = pearsonr(df["volume_traded"], df["trades_count"])
print('Pearsons correlation: %.3f' % corr)

As we can see, there is a strong correlation between `volume_traded` and `trades_count`. This is quite logical. Now, I'll see if `trades_count` or `volume_traded` is correlated with `price_close`.

In [ ]:
corr, _ = pearsonr(df["trades_count"], df["price_close"])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(df["volume_traded"], df["price_close"])
print('Pearsons correlation: %.3f' % corr)

`trades_count` and `price_close` are correlated (to some extent). This means more trades are done when the close price is high. However, there is no correlation between `volume_traded` and `price_close`, meaning the volume traded is not realted with the close price, even though the higher the close price the more trades are done.

## Q3: Covid cases and Bitcoin price

## Load and Check Data

Now, I'll try to see if there was any correlation between the covid cases (both daily and cumulative) and Bitcoin price. First, I read the csv file with Covid cases data into a dataframe.

In [ ]:
covid_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

Next, I check the dataset.

In [ ]:
# Check dataset shape
covid_df.shape

In [ ]:
# Check the top of the data
covid_df.head()

In [ ]:
# Check the bottom of the data
covid_df.tail()

In [ ]:
# Check the number of rows in the dataframe
len(covid_df)

In [ ]:
# Check null values
covid_df.isnull().sum()

I won't use the `Province/State` column, so no need to impute or drop nulls. Since I will only need the daily Covid cases worldwide for 2020, I will only keep the columns with 2020 data and combine the cases from all countries.

In [ ]:
# Columns with 2020 data
date_cols = [x for x in covid_df.columns if x[-3:] == '/20']

In [ ]:
# Keep the columns with 2020 data
covid_df = covid_df[date_cols]

In [ ]:
# Check the dataset
covid_df

In [ ]:
# Convert the column names to datetime objects
date_cols = pd.to_datetime(covid_df.columns, format="%m/%d/%y")
covid_df.columns = date_cols

In [ ]:
# Check the dataset
covid_df

Since I will need the total cumulative cases and new daily cases, I combine the given data per country and store them in a dataframe.

In [ ]:
# Combine the covid cases from all countries
covid_cases = pd.DataFrame(covid_df.sum(), columns=['cases'])

In [ ]:
# Check the dataset
covid_cases

In [ ]:
# Add a column with daily new cases
covid_cases['daily_cases'] = covid_cases.cases - covid_cases.cases.shift(1, fill_value=0)

In [ ]:
# Check the dataset
covid_cases

Then, I need to merge the Bitcoin price and Covid case dataframes. To do so, I'll create a new column in each dataframe with the day of year, to later merge on. 

In [ ]:
df['dayofyear'] = df.time_open.dt.dayofyear

In [ ]:
covid_cases['dayofyear'] = covid_cases.index.dayofyear

In [ ]:
# Merge on dayofyear
merged_df = pd.merge(df, covid_cases, on='dayofyear').drop('dayofyear', axis=1)

In [ ]:
# Check the dataset
merged_df

Now, I'll plot the Bitcoin prices and # of Covid cases against each other.

In [ ]:
# Price and Covid cases over time
ax = sns.lineplot(x="time_open", y="price_close", data=merged_df, color='#FFBC42')
ax_sec = ax.twinx()    
ax_sec = sns.lineplot(x="time_open", y="cases", data=merged_df, color='#748B75')

# Set y-axis label
ax.set_ylabel("Price ($)")
ax_sec.set_ylabel("Cases")

# Set x-axis label
ax.set_xlabel("Date")

# Title
ax.set_title("BTC Price and # of Covid Cases Over Time")

# Show the plot
plt.show()

There seems to be a correlation between these variables. We can check that by conducting Pearson's correlation test.

In [ ]:
corr, _ = pearsonr(merged_df.price_close, merged_df.cases)
print('Pearsons correlation: %.3f' % corr)

The result ensures that there is a strong correlation between the # of cumulative Covid cases and Bitcoin prices. Next, I'll plot the daily new cases agains Bitcoin prices.

In [ ]:
# Price and New Daily Covid cases over time
ax = sns.lineplot(x="time_open", y="price_close", data=merged_df, color='#FFBC42')
ax_sec = ax.twinx()    
ax_sec = sns.lineplot(x="time_open", y="daily_cases", data=merged_df, color='#748B75')

# Set y-axis label
ax.set_ylabel("Price ($)")

# Set x-axis label
ax.set_xlabel("Date")

# Title
ax.set_title("BTC Price and # of New Covid Cases Over Time")

# Show the plot
plt.show()

Again, there seems to be a correlation between the two variables, though not as strong as between the last two variables. I'll conduct a Pearson's correlation test to find the correlation coefficient.

In [ ]:
corr, _ = pearsonr(merged_df.price_close, merged_df.daily_cases)
print('Pearsons correlation: %.3f' % corr)

As can be seen, there, indeed, is a strong correlation between the new Covid cases and Bitcoin price.

## Write-up

For Module Assignment 3, I have chosen to get the Bitcoin price data for 2020 using the CoinAPI. The dataset contains hourly Bitcoin price data, including open, close, low, and high prices. There is also data on trades count and volume traded. I've chosen 3 questions, that I will try to answer using the data I have. Firstly, I want to assure the time series data of Bitcoin prices is non-stationary, i.e., the means, variances, and covariances change over time. This also means that it is hard to model the data. As many different factors affect Bitcoin price, its market is quite volatile, and there are rapid and significant increases or decreases, I know it should be non-stationary. Secondly, I try to find out if there is a correlation between `volume_traded` or `trades_count` and `price_close` variables. I believe that when the Bitcoin price is high, more trades are done. Nonetheless, I am not really sure about `volume_traded` at high prices. And lastly, I'll try to see if there is a correlation, between Covid cases (both daily new cases and total cumulative cases) and the Bitcoin price (or if there was a correlation in 2020). For the last question, I would need another dataset, which I'll describe in detail as I answer Question 3.

It is important to mention that CoinAPI has both a daily limit and a limit on data received by a request, which makes using it (the free version) a bit inconvenient. Thus, I had to send multiple requests and combine the received data. After gathering and combining the data for the whole interval, I found out some rows were missing. I tried requesting these missing values again, however, it turned out the mistake was probably in the API since the request returned an empty list. Nonetheless, out of 8784 entries (24 hours * 366 days), only 25 were missing, which is roughly 0.3% and is negligible even though the data is time series (there were not many consecutive missing values). After collecting the data, I checked it and made sure the interval is correct, checked the top and bottom, and proceeded to Question 1.

To find out whether the time series is stationary or non-stationary, I conducted a Dickey-Fuller Test, which uses the following null and alternative hypotheses:
- H0: The time series is non-stationary. In other words, it has some time-dependent structure and does not have constant variance over time.
- HA: The time series is stationary.
If the p-value from the test is less than some significance level (I'll take 0.05), then we can reject the null hypothesis and conclude that the time series is stationary. I performed the test using the `adfuller` function from `statsmodels.tsa.stattools` library. The p-value returned was much higher than 0.05, which means we don't have enough evidence to reject the Null hypothesis, so we accept it, meaning the time series is non-stationary. This lines up with my assumption.

Next, I've tried to see if any of `volume_traded` or `trades_count` is correlated with `price_close`. I've tested that using the Pearson correlation test, the `pearsonr` function from the `scipy.stats` library. However, firstly, I calculated the correlation between `volume_traded` and `trades_count`. These two variables are strongly positively correlated (ρ = .832), meaning that the larger the volume traded is, the more trades have been taken place, which is logical. Next, I calculated the Pearson test between the remaining two pairs. It turned out that while the there is some positive correlation between `volume_traded` and `price_close` (ρ = .443), there is no correlation between `trades_count` and `price_close` (ρ = .019). This means that the higher the Bitcoin price is, the more is the volume traded, however, it does not mean that more trades have occurred.

For the last question, I got another dataset from John Hopkins University's Covid tracking website (via GitHub). The dataset contains cumulative daily cases for different countries and regions from January 22, 2020, up till now. However, I only need the total number of cases per day, and only the data for 2020, so I created a smaller dataframe with only the combined Covid cases per day. Then I added another column, that had the daily new cases. I plotted both the Bitcoin price and the daily cumulative Covid cases on the same chart but with different scales. It could be seen visually on the chart that the two variables are positively correlated. I did a Pearson correlation test and found out that there is, indeed, a strong positive correlation between them (ρ = .924). I did the same with Bitcoin price and daily new Covid cases, and again found out that the two are positively correlated (ρ = .840). However, correlation does not mean causation.

All in all, I found that the Bitcoin price time series is non-stationary, that `volume_traded` is positively correlated with `trades_count` and `price_close`, and that the number of Covid cases (both new and cumulative) and the Bitcoin price are correlated. Again, the analysis was done on the 2020 Bitcoin price data taken from Binance. Also, there might have been a bunch of other factors that affected the Bitcoin price.

## Q0: Difference Between Consecutive Weeks

How can I find an answer to this question? Can I use t-tests ona time series (values are dependent)?

First, I'll divide the data into weeks. To do that, I create a new column in the dataframe, called `week`.

In [ ]:
# Add new column by binning
df['week'] = pd.cut(df.time_open.dt.dayofyear, bins=range(0, 373, 7), labels=range(1, 54))

In [ ]:
# Check how many entries every week has
df.week.value_counts().sort_index()

I will now plot a bar graph, to see the mean price for each week.

In [ ]:
# Barplot for average close price per week
ax = sns.barplot(x="week", y="price_close", data=df, palette = ['#748B75'])

# Some x tick labels are too long, so, I need to wrap the title
# Get the x tick labels
ax.get_xticklabels()

# Get the text out
texts = [t.get_text()  for t in ax.get_xticklabels()]

# Wrap the text
texts = [textwrap.fill(t.get_text(), 12)  for t in ax.get_xticklabels()]

# Set the new wrapped titles
ax.set_xticklabels([textwrap.fill(t.get_text(), 12)  for t in ax.get_xticklabels()])


# Set y-axis label
ax.set_ylabel("Price ($)")

# Set x-axis label
ax.set_xlabel("Week")

# Title
ax.set_title("Average Price Per Week")

# Show the plot
plt.show()

I will save the dataframe, so that I don't have to get it using the API again.

In [ ]:
# Save the dataframe to a csv file
df.to_csv("BTCUSDT_2020.csv")

Now, I will conduct t-tests for the price data for each two consecutive weeks. I will also plot the pricelines. First, I define a plotting function.

In [ ]:
def plot_priceline(start_week, end_week, pvalue):
    """
    Plots the priceline for the given interval (in weeks).
    
    Keyword parameters:
        start_week - the start of the interval.
        
        end_week   - the end of the interval.
        
        pvalue     - the pvalue of the t-test for the given two weeks.    
    """
    # Priceline
    ax = sns.lineplot(x="time_open", y="price_close", data=df, color='#748B75' if pvalue < 0.05 else '#EE2E31')
    # Set y-axis label
    ax.set_ylabel("Price ($)")

    # Set x-axis label
    ax.set_xlabel("Date")

    df_weeks = df[(week <= df.week) & (df.week <= week+1)]
    
    # Set x limits
    limits = list(df_weeks.time_open)
    ax.set_xlim(limits[0], limits[-1])
    
    # Set y limits
    ax.set_ylim(df_weeks.price_close.min()*.95, df_weeks.price_close.max()*1.05)

    # Title
    ax.set_title(f"BTC Price Over Time | Weeks {week}-{week+1} | p-value: {pvalue:.5f}")

    # Show the plot
    plt.show()

In [ ]:
# For each two consecutive weeks, conduct a t-test
for week in range(1, 53):
    prev_week = df[(df.week == week)].price_close
    next_week = df[(df.week == week + 1)].price_close

    res = stats.ttest_ind(prev_week, next_week)
    
    plot_priceline(week, week+1, res.pvalue)
    plt.close()

Almost every t-test returned a small enough p-value, which means there is significant evidence to reject the Null hypothesis and accept that there is dependence between date (time) and price. Since this is a timeseries and each value is dependent on the previous price, this is logical. Also, many other factors affect the price at different points of time, meaning there is dependance between time and price.